In [1]:
import numpy as np
import os
dtype = np.float32

import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm
import os

from transformers import CLIPVisionModelWithProjection, AutoModelForCausalLM, LlamaForCausalLM
from transformers import AutoModel, AutoTokenizer, OPTForCausalLM, BloomForCausalLM
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

import glob
import random
import json
import os

from datasets import load_dataset

import functools
import gc
from collections import defaultdict
from typing import List

import torch
import torch.nn as nn
import tqdm
# from tinychat.models import LlavaLlamaForCausalLM
from transformers.models.bloom.modeling_bloom import BloomForCausalLM
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from transformers.models.opt.modeling_opt import OPTForCausalLM

import numpy as np
from scipy.linalg import eigh

def topk_eigenvectors(A, k):
    # A는 대칭 행렬이어야 합니다.
    # k는 추출할 최상위 고유벡터의 수입니다.
    
    # 고유값과 고유벡터 계산
    # 'LA'는 가장 큰 고유값을 의미
    eigenvalues, eigenvectors = eigh(A, subset_by_index=(A.shape[0] - k, A.shape[0] - 1))
    
    # 고유벡터의 노름이 1인지 확인
    norms = np.linalg.norm(eigenvectors, axis=0)
    # print("Norms of eigenvectors:", norms)
    
    assert np.allclose(norms, 1)
    # 고유벡터의 노름이 1에 가까운지 확인
    # if np.allclose(norms, 1):
    #     print("All eigenvectors are normalized.")
    # else:
    #     print("Some eigenvectors are not normalized.")
        
    eigenvectors
    
    return eigenvalues, eigenvectors.T

/home/jgryu/miniconda3/envs/nic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Cal Hess topk eigens

In [2]:
model_list = [
    # 'meta-llama/Meta-Llama-3-8B',
    # 'meta-llama/Llama-2-7b-hf',
    # 'meta-llama/Llama-2-13b-hf',
    # 'lmsys/vicuna-7b-v1.5',
    # 'lmsys/vicuna-13b-v1.5',
    'facebook/opt-6.7b',
]

n_samples=128
seqlen=512
calib_data="pileval"
batch_size = 12
rank = 10

# top = np.array([0.1,1,10])
# qlevel = [3,2,1]

for model_name in model_list:
    
    model_name = model_name.replace('/', '--')
    print('model_name: ', model_name)

    model_path = f"./hf_model/{model_name}"

    save_path = f'./hessian/{model_name}'
    os.makedirs(save_path, exist_ok=True)
    hess  = torch.load(save_path + f'/{calib_data}_n_samples{n_samples}_seqlen{seqlen}.pt')
    
    hess_eigen = []
    for i in tqdm.tqdm(range(len(hess))):
        
        eigen_one_layer = defaultdict(list)
        
        for n, h in hess[i].items():
            # print(i, n, h.shape)
            
            eigen = {}
            
            eigenvalues, eigenvectors = topk_eigenvectors(h, rank)
            
            eigen['eigenvalues'] = torch.from_numpy(eigenvalues)
            eigen['eigenvectors'] = torch.from_numpy(eigenvectors)
            
            # print(eigen['eigenvalues'].shape)
            # print(eigen['eigenvectors'].shape)
            
            eigen_one_layer[n] = eigen
        
        hess_eigen.append(eigen_one_layer)
    
    torch.save(hess_eigen, save_path + f'/{calib_data}_n_samples{n_samples}_seqlen{seqlen}_rank{rank}_eigen.pt')
    

model_name:  facebook--opt-6.7b


/tmp/ipykernel_3312821/1584214709.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hess  = torch.load(save_path + f'/{calib_data}_n_samples{n_samples}_seqlen{seqlen}.pt'

100%|██████████| 32/32 [31:41<00:00, 59.41s/it]
